In [17]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [18]:
dataset = pd.read_csv('../Datasets/IMDB_Dataset.csv')
print(dataset.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [19]:
clean_dataset = dataset.replace('positive',1).replace('negative',0)
print(clean_dataset.head())

                                              review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1


In [20]:
splitValue = 0.8

train = clean_dataset.sample(frac=splitValue, ignore_index=True)
validation = clean_dataset.drop(train.index)
train_labels = np.array(train['sentiment'])
validation_labels = np.array(validation['sentiment'])

In [35]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='OOV')
tokenizer.fit_on_texts(clean_dataset['review'])
train_sequences = tokenizer.texts_to_sequences(train['review'])
validation_sequences = tokenizer.texts_to_sequences(validation['review'])

In [36]:
padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=120, truncating='post')
validation_padded = tf.keras.preprocessing.sequence.pad_sequences(validation_sequences, maxlen=120, truncating='post')

In [37]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(200000, 16, input_length=120),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [38]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
model.fit(padded, train_labels, epochs=10, validation_data=(validation_padded, validation_labels))

Epoch 1/10
1250/1250 [==============================] - 12s 9ms/step - loss: 0.4387 - accuracy: 0.7859 - val_loss: 0.2360 - val_accuracy: 0.9122
Epoch 2/10
1250/1250 [==============================] - 14s 11ms/step - loss: 0.1751 - accuracy: 0.9365 - val_loss: 0.1183 - val_accuracy: 0.9603
Epoch 3/10
1250/1250 [==============================] - 17s 13ms/step - loss: 0.0404 - accuracy: 0.9907 - val_loss: 0.1002 - val_accuracy: 0.9675
Epoch 4/10
1250/1250 [==============================] - 15s 12ms/step - loss: 0.0075 - accuracy: 0.9990 - val_loss: 0.1102 - val_accuracy: 0.9671
Epoch 5/10
1250/1250 [==============================] - 15s 12ms/step - loss: 0.0015 - accuracy: 0.9999 - val_loss: 0.1216 - val_accuracy: 0.9664
Epoch 6/10
1250/1250 [==============================] - 14s 12ms/step - loss: 4.2335e-04 - accuracy: 1.0000 - val_loss: 0.1312 - val_accuracy: 0.9667
Epoch 7/10
1250/1250 [==============================] - 16s 13ms/step - loss: 1.7314e-04 - accuracy: 1.0000 - val_loss: 0

In [ ]:
def predict(sentence):
    new_sequence = tokenizer.texts_to_sequences([sentence])
    new_padded = tf.keras.preprocessing.sequence.pad_sequences(new_sequence, maxlen=120, truncating='post')

    prediction = model.predict(new_padded)

    sentiment_score = prediction[0][0]
    sentiment_label = "positive" if sentiment_score >= 0.5 else "negative"

    print(f"Sentence: {sentence}")
    print(f"Sentiment: {sentiment_label} (Score: {sentiment_score:.4f})")

predict(input())
